<a href="https://colab.research.google.com/github/SuchandanG/Quantum-Secret-Sharing/blob/main/Partitioning_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter

def compute_mu_E(Gamma_0, n):
    """Compute essentialness measure μ_E for each participant."""
    mu_E = Counter()
    for subset in Gamma_0:
        for participant in subset:
            mu_E[participant] += 1
    for i in range(1, n + 1):
        mu_E[i] += 0  # Ensure all participants appear
    return mu_E

def partition_gamma(Gamma_0, n):
    partitions = []
    Gamma_0 = [set(g) for g in Gamma_0]  # Ensure input is in set format
    j = 1

    while len(Gamma_0) > 2:
        mu_E = compute_mu_E(Gamma_0, n)
        min_mu = min(mu_E.values())
        S_E = {p for p in mu_E if mu_E[p] > min_mu}  # exclude strictly minimal
        print(S_E)
        while True:
            if not S_E:
                partitions.append(Gamma_0.copy())
                return partitions

            Gamma_prime = [g for g in Gamma_0 if S_E.issubset(g)]

            if len(Gamma_prime) == 0:  # Case (i)
                # Remove a participant with minimal μ_E from S_E
                min_S_E = min(mu_E[p] for p in S_E)
                for p in list(S_E):
                    if mu_E[p] == min_S_E:
                        S_E.remove(p)
                        break
            elif len(Gamma_prime) == 1:  # Case (ii)
                min_S_E = min(mu_E[p] for p in S_E)
                for p in list(S_E):
                    if mu_E[p] == min_S_E:
                        S_E.remove(p)
                        break
            else:  # Case (iii)
                Gamma_0j = Gamma_prime.copy()
                # Check near-disjointness (at most one overlapping pair)
                def target_set(g): return g - S_E
                overlaps = 0
                for i in range(len(Gamma_0j)):
                    for k in range(i + 1, len(Gamma_0j)):
                        if target_set(Gamma_0j[i]) & target_set(Gamma_0j[k]):
                            overlaps += 1
                if overlaps <= 1:
                    partitions.append(Gamma_0j)
                else:
                    # Remove sets greedily to ensure near-disjointness
                    while overlaps > 1 and Gamma_0j:
                        Gamma_0j.pop()
                        overlaps = 0
                        for i in range(len(Gamma_0j)):
                            for k in range(i + 1, len(Gamma_0j)):
                                if target_set(Gamma_0j[i]) & target_set(Gamma_0j[k]):
                                    overlaps += 1
                    partitions.append(Gamma_0j)
                # Update Gamma_0
                Gamma_0 = [g for g in Gamma_0 if g not in Gamma_0j]
                break
    if Gamma_0:
        partitions.append(Gamma_0)
    return partitions

# Input
Gamma_0_input = [{1, 2, 3}, {1, 2, 4}, {1, 2, 5, 6}]
n_participants = 6

# Run algorithm
partitions_result = partition_gamma(Gamma_0_input, n_participants)
partitions_result

{1, 2}


[[{1, 2, 3}, {1, 2, 4}, {1, 2, 5, 6}]]